<a href="https://colab.research.google.com/github/MrDarkTesla/QCL_2021/blob/main/QCL_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install qiskit

     |████████████████████████████████| 8.5MB 5.8MB/s 
     |████████████████████████████████| 17.6MB 236kB/s 
     |████████████████████████████████| 204kB 38.9MB/s 
     |████████████████████████████████| 204kB 40.8MB/s 
     |████████████████████████████████| 2.1MB 40.3MB/s 
     |████████████████████████████████| 1.0MB 35.7MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 194kB 40.1MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 583kB 34.7MB/s 
     |████████████████████████████████| 3.2MB 35.2MB/s 
     |████████████████████████████████| 5.5MB 37.9MB/s 
  Created wheel for qiskit: filename=qiskit-0.24.0-cp37-none-any.whl size=2918 sha256=6a761b88e117e4f8a0c250abe190f3482db0d1858330a5c70a676b74e655af7c
  Stored in directory: /root/.cache/pip/wheels/22/fe/0a/26b7a409d1a9c3698b0319c7d009a97bffaae392d755158dca
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2

In [ ]:
# Initialization
import matplotlib.pyplot as plt
import numpy as np

# Importing Qiskit
from qiskit import IBMQ, BasicAer, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# Import basic plot tools
from qiskit.tools.visualization import plot_histogram

# Scipy
from scipy.optimize import rosen, differential_evolution

# Files
from google.colab import files

In [ ]:
def f1(x):
  return x ** 2 + 1


def f2(x):
  return x ** 2 + x ** 3


def f3(x):
  return np.sin(x + 1)


def f4(x):
  return np.exp(x) + x


TEACHER = np.random.choice(np.linspace(-1, 1), 10)

In [27]:
X = np.random.rand(50, 2)

def f(x):
  if x[0] >= 0.5:
    return 1
  return 0



In [28]:
def U_in(x, xq, qc):
  phi = (x[0]) * np.pi * 2
  omega = (x[1] - 0.5) * np.pi
  qc.rx(omega, xq)
  qc.rz(phi, xq)


def U_out(i, j, xq, qc, theta, n):
  t = (i + j * n) * 3
  qc.rx(theta[t + 0], xq[i])
  qc.rz(theta[t + 1], xq[i])
  qc.rx(theta[t + 2], xq[i])


def Hamelton(n, qc):
  for i in range(n - 1):
    qc.cx(i, i + 1)
#  for i in range(n - 1, 0, -1):
#    qc.cx(i, i - 1)


def Bfunk(n, iter, count):
  b = [0] * n
  for k in count.keys():
    for i, l in enumerate(k):
      b[i] += int(l) * count[k]
  for i in range(n):
    b[i] /= iter
  return b



In [29]:
def fi(xi, N, deapth, theta, Iterations, Omega):

  xq = QuantumRegister(N, "x")
  yc = ClassicalRegister(1, "y") # N instead
  qc = QuantumCircuit(xq, yc)

  U_in(xi, xq, qc) 
  for j in range(deapth):
    Hamelton(N, qc)
    for k in range(N):
      U_out(k, j, xq, qc, theta, N)

  qc.measure(xq[0], yc)
  job = execute(qc, backend, shots=Iterations)
  result = job.result()
  count = result.get_counts()
  B = Bfunk(N, Iterations, count)
  Y = np.dot(B, Omega)
  return sum([float(k)*count[k] for k in count]) / Iterations 
  #(Y + 1) / 2 + 1

In [31]:
def Lfunk(Theta):
  s = np.pi /2
  Omega = [1/3, 1/3, 1/3]
  Iterations = 1000 
  deapth = 2   
  N = 3 
  L = 0 #np.dot(Omega, Omega) 
  for t in X: # was TEACHER
    L += (t - fi(t, N, deapth, theta, Iterations, Omega)) ** 2
  return L


def GradLfunk(theta):
  gr = [0] * len(theta)
  s = np.pi /2
  Omega = [1/3, 1/3, 1/3]
  Iterations = 1000 
  deapth = 2   
  N = 3 
  for t in X: # was TEACHER
    for i in range(len(theta)):
      theta[i] += s
      A = fi(t, N, deapth, theta, Iterations, Omega)
      theta[i] -= 2 * s
      B = fi(t, N, deapth, theta, Iterations, Omega)
      theta[i] += s
      C = (A - B)
      gr[i] += (fi(t, N, deapth, theta, Iterations, Omega) - f(t)) * C / np.sin(s)
  return np.array(gr)


In [32]:
def ftestgrad(x):
  return np.array([2 * (x[0] - 1), 2 * (x[1] + 1)])


In [33]:
backend = Aer.get_backend('qasm_simulator')
theta = np.zeros(3 * 2 * 3)
bounds = [(0,2 * np.pi)] * len(theta)

etta = .01
for i in range(10):
  theta -= etta * GradLfunk(theta)
  print(Lfunk(theta))


[8.19497822 9.67484721]
[8.22484169 9.65546925]
[8.29279942 9.68255506]
[8.26415403 9.74164783]
[8.25556783 9.51680487]
[8.15979658 9.57531543]
[8.0632969  9.40961027]
[8.22400725 9.20307545]
[8.02895409 8.84234872]
[7.86322803 8.76645134]


In [ ]:
backend = Aer.get_backend('qasm_simulator')
result = differential_evolution(Lfunk, bounds)
np.save("Result.npy", result)
files.download('Test.npy')

RuntimeError: ignored